In [1]:
'''uncomment if you wish to run NN on CPU'''

# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

'uncomment if you wish to run NN on CPU'

In [2]:
from numpy import *
from numpy import linalg as la
from scipy import linalg as sa
from scipy import special as ss
from scipy.special import factorial

import matplotlib.pyplot as plt

from scipy.optimize import minimize

import time as time


In [3]:
import keras
from keras import optimizers
from keras import initializers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Conv2D, Flatten, Reshape, AveragePooling2D,UpSampling1D
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [121]:
def randompure(dim,n):
    rpure = random.normal(0,1,[dim,n]) + 1j*random.normal(0,1,[dim,n])
    rpure = rpure/la.norm(rpure,axis=0)
    rhon = array([dot(rpure[:,[i]],rpure[:,[i]].conjugate().transpose())  for i in range(n)])
    return rhon
    
def randomHS(dim):
    matG = random.normal(0,1,[dim,dim]) + 1j*random.normal(0,1,[dim,dim])
    rho = (matG@matG.transpose().conjugate())/trace(matG@matG.transpose().conjugate())
    return rho

def herbasis(dim):
    pom1 = zeros([1,dim,dim])+1j*zeros([1,dim,dim])
    pom1[0] = identity(dim)
    arrays = [dot(transpose(pom1[0][[i]]),pom1[0][[i]]) for i in range(dim-1)]
    pom = stack(arrays,axis=0)
    her = concatenate((pom1,pom),axis=0)
    arrays = [dot(transpose(her[0][[i]]),her[0][[j]])+dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
    pom = stack(arrays,axis=0)
    her = concatenate((her,pom),axis=0)
    arrays = [-1j*dot(transpose(her[0][[i]]),her[0][[j]])+1j*dot(transpose(her[0][[j]]),her[0][[i]]) for i in range(dim) for j in range(i+1,dim)]
    pom = stack(arrays,axis=0)
    pom = concatenate((her,pom),axis=0)
    return pom

def gellmann(Q,dim):
    q = zeros([dim**2,dim,dim])+1j*zeros([dim**2,dim,dim])
    for i in range(dim**2):
        v = Q[i]
        for j in range(0,i):
            v = v-trace(v@q[j])*q[j]
        q[i] = v/sqrt(trace(v@v))
    q[0] = identity(dim)
    return q

def blochFromRho(rho,A):
    l = shape(A)[0]
    return array([real(trace(rho@A[i])) for i in range(l)])

def rhoFromBloch(blochP,A):
    global dim
    l = shape(A)[0]
    return 1/dim*(identity(dim)+sum([blochP[i]*A[i] for i in range(l)],axis=0))

def rhoFromBlochG(blochP,A):
    global dim
    l = shape(A)[0]
    return 1/dim*(sum([blochP[i]*A[i] for i in range(l)],axis=0))

def srm(m):
    global dim
    mstates = randompure(dim,m)
    mstates = reshape(mstates, (m,dim,dim))
    Sop = sum(mstates,axis=0)
    Sop = sa.sqrtm(la.inv(Sop))
    pom = [(Sop@mstates[i])@Sop for i in range(m)]
    return pom

def probdists(stav,povm):
    l = shape(povm)[0]
    probtrue = array([real(trace(stav@povm[i])) for i in range(l)])
    probtrue = probtrue/sum(probtrue)
    return probtrue

def Cmat(pom,G):
    global dim
    l = shape(pom)[0]
    matC = zeros((shape(pom)[0],dim**2))
    for m in range(l):
        for n in range(dim**2):
            matC[m,n]=trace(pom[m]@G[n])
    return 1/dim*matC

def choFromRho(rho):
    noStates = shape(rho)[0]
    choVec = zeros((noStates,dim**2))
    for n in range(noStates):
        L = la.cholesky(rho[n])
        imL = imag(L)
        reL = real(L)
        k_i = 0
        for i in range(dim):
            for j in range(0,i+1):
                choVec[n,k_i] = reL[i,j]
                k_i+=1

        for i in range(1,dim):
            for j in range(i):
                choVec[n,k_i] = imL[i,j]
                k_i+=1
    return choVec

def rhoFromCho(choVec):
    rhoC = zeros((dim,dim),complex)
    k_i=0
    for i in range(dim):
        for j in range(0,i+1):
            rhoC[i,j] += choVec[k_i]
            k_i+=1

    for i in range(1,dim):
        for j in range(i):
            rhoC[i,j] += 1j*choVec[k_i]
            k_i+=1
    return dot(rhoC,rhoC.T.conj())/trace(dot(rhoC,rhoC.T.conj()))

def MLalg(data, rhoinit,povm,epsilon,stop):
    global dim
    rho = rhoinit
    trdist = 1
    k = 0
    pomS = zeros([dim,dim], complex)
    for n in range(shape(povm)[0]): pomS = pomS + povm[n]
    Gop = sa.pinv(sa.fractional_matrix_power(pomS,1/2))
    while trdist > epsilon and k<stop:
        R = zeros([dim,dim], complex)
        prob = probdists(rho,povm)
        for i in range(shape(povm)[0]):
            if data[i]>10**(-10):
                R += data[i]/prob[i]*povm[i]
        rhonew = dot(dot(Gop,dot(R,rho)),dot(R,Gop))
        rhonew = rhonew/trace(rhonew)
        trdist = trace(dot((rho-rhonew),(rho-rhonew)))
        rho = rhonew
        k+=1
    return rho

def HSdist(A,B):
    return trace((A-B)@(A.conjugate().transpose()-B.conjugate().transpose()))

In [187]:
'''dimension of the Hilbert space that we choose to work in'''

dim = 3

In [188]:
'''create hermitian basis in a dim-dimensional HS'''

Q = herbasis(dim)
GAll = gellmann(Q,dim)*sqrt(dim)
G = GAll[1::]
GAll[0] = identity(dim)

In [189]:
'''load random projectors in a given dimension or create your own measurement'''
'''however, botch Bloch and Cholesky NNs works only with our prechosen measurement settings'''

file=load('measurements/SRMpom'+str(dim**2)+'.npz')
pom=file['arr_0']

'''generate the masurement matrix C'''
cmat = Cmat(pom,GAll)

C:\Users\Dominik\AppData\Local\Temp\ipykernel_19704\1594535297.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  matC[m,n]=trace(pom[m]@G[n])


In [190]:
'''get a quantum state'''

rho = [ randomHS(dim) ]

'''get the Cholesky vector and the Bloch parameters'''

choV = choFromRho(rho)

bL = blochFromRho(rho[0],GAll)/sqrt(dim-1)

'''and a corresponding propability distribution'''

probsTrue = probdists(rho[0],pom)

print(probsTrue.sum())

1.0


In [191]:
'''sample the probability distribution'''

nSamples = 500
probsExp = random.multinomial(nSamples,probsTrue)/nSamples

print(probsExp.sum())

1.0


In [192]:
'''choose if you wish to work with a true probability distribution or a sampled one'''

# probD = probsTrue
probD = probsExp

In [193]:
'''linear inversion'''

blochLI = dot(la.pinv(cmat),probD.transpose())

rhoLI = rhoFromBlochG(blochLI,GAll)

print(HSdist(rho[0], rhoLI))

(0.6533052676575583+0j)


In [194]:
'''load models'''

modelCho = load_model('models/bestModelCho'+str(dim**2)+'.h5')
modelBloch = load_model('models/bestModelBloch'+str(dim**2)+'.h5')

In [195]:
'''reshape the probability vector'''

x_test = reshape(probD,(1,dim**2))

In [196]:
'''get predictions of the respective representations of the quantum state'''

blochPredicted = modelBloch.predict(x = x_test)*sqrt(dim-1)

choPredicted = modelCho.predict(x = x_test)

In [197]:
'''get the density matrices and calculate the Hilbert Schmidt distance'''

rhoB = rhoFromBlochG(blochPredicted[0],GAll)

print(HSdist(rhoB,rho[0]))

rhoC = rhoFromCho(choPredicted[0])

print(HSdist(rhoC,rho[0]))

(0.03304744509757359+0j)
(0.2113886562555401+0j)


In [198]:
'''semi-definite program'''

import cvxpy as cp

In [199]:
x = cp.Variable(dim**2)
constraints = [rhoFromBlochG(x,GAll)>>0]
constraints += [cp.trace(rhoFromBlochG(x,GAll))==1]
prob = cp.Problem(cp.Minimize(cp.sum_squares((cmat@x)-probD)),constraints)
prob.solve()
solSDP = x.value

In [200]:
'''prediction made by a semi-definite program'''

rhoSDP = rhoFromBlochG(solSDP, GAll)

print(HSdist(rhoSDP,rho[0]))

(0.06439552885315726+0j)


In [201]:
'''maximum likelihood method'''

rhoML = MLalg(probD,identity(dim)/dim,pom,10**(-14),10**5)

print(HSdist(rhoML,rho[0]))

(0.049451839403573034+0j)


In [202]:
'''comparision of errors'''

print("the HS distance between true and estimated QS by a LI", HSdist( rho[0], rhoLI))
print("--------------------------------------------- by a Bloch NN", HSdist( rho[0], rhoB))
print("--------------------------------------------- by a Cholesky NN", HSdist( rho[0], rhoC))
print("--------------------------------------------- by a SDP", HSdist( rho[0], rhoSDP))
print("--------------------------------------------- by a ML", HSdist( rho[0], rhoML))


the HS distance between true and estimated QS by a LI (0.6533052676575583+0j)
--------------------------------------------- by a Bloch NN (0.03304744509757359+0j)
--------------------------------------------- by a Cholesky NN (0.2113886562555401+0j)
--------------------------------------------- by a SDP (0.06439552885315726+0j)
--------------------------------------------- by a ML (0.049451839403573034+0j)
